# Grab land cover for specified extent
Say you want to grab land cover data for a specific location, but you don't want to download the entire NLCD dataset to do that. The ArcGIS Python API can help!

Here I present an example of doing just that. The steps involve:
* Prepping for the analysis: importing modules and authenticating our arcgis session
* Locating the land cover data **item** and creating a **imagery layer** from it
* 

*Resource: https://developers.arcgis.com/python/guide/raster-analysis-advanced-concepts/*

### Prepping for analysis: importing and authenticating
* Import the the arcgis `GIS` module. We're going to do some geocoding so we need to create the arcgis `geocoding` module as well. Lastly, enable the Jupyter `display` object

In [ ]:
#Import the GIS object.  and display modules
from arcgis import GIS
#from arcgis.geocoding import geocode
from IPython.display import display, Image

* Authenticate our GIS object using our ArcGIS Pro account

In [ ]:
#Create the GIS object, authenticating with your ArcGIS Pro account
gis = GIS('pro')

### Searching for and accessing the content
What we want is 2011 NLCD data, provided as an *image service* (i.e. as a raster). You could search for the data from within ArcGIS Pro or via the [ArcGIS Online](http://www.arcgis.com) website, but we'll do it right here. 

Like any web search, it's a bit of an art knowing how best to locate the resource you want. At play are what general search keywords to include, and specific categories like `owner` or `item-type` to invoke. We do, however, want to search outside the Duke Community, we we want to include `outside_org=True`. 

I've decided to use `NLCD 2011` as a general search term, filter results for only those that `esri` provides, and limit results to image services: 

In [ ]:
#Search for land cover and print the number of results
nlcd_results = gis.content.search("NLCD 2011, owner:esri",   #The query used to select records
                                  item_type='Image Service', #Filter just those that are Imagery layers
                                  max_items=100,             #Default is 10 items; let's look for more
                                  outside_org=True)          #Default is to just search Duke's portal; turn that off
#Reveal how many hits we got
len(nlcd_results)

► This gives us several results, enough to show a list...

In [ ]:
#Show a complete list of results
nlcd_results

* The *second* item is the one we want. Let's store that as a variable named `lc_item`. 

In [ ]:
#Get the second result and view what kind of Python object it is. 
nlcd_item = nlcd_results[1]
type(nlcd_item)

In [ ]:
#Show what kind of ArcGIS object it is
nlcd_item.type

In [ ]:
#Show what the item's ID is 
nlcd_item.id

In [ ]:
#Show the item info
nlcd_item

## A second option for finding and fetching data: ESRI's Living Atlas
ESRI's [Living Atlas](https://livingatlas.arcgis.com) contains a vast array of quality controlled data. Here's how we can access images hosted there: 
* Open the Living Atlas in your web browser. 
* Search for NLCD and select the [USA NLCD Land Cover](https://www.arcgis.com/home/item.html?id=3ccf118ed80748909eb85c6d262b426f)
* Note the Item ID in the page's URL: 3ccf118ed80748909eb85c6d262b426f
* Fetch the imagery layer using this ID:

In [ ]:
#Fetch the NLCD image layer service using its item ID
nlcd_item2 = gis.content.get('3ccf118ed80748909eb85c6d262b426f')
nlcd_item2.type

In [ ]:
nlcd_item2

### Accessing the <u>layers</u> contained in the image service
Image Services are a container for individual image layers. So to access the actual raster datasets we want to analyze or download, we need to drill into the service an pull out individual layers. 

In [ ]:
#Reveal the layers associated with this service
nlcd.layers

In [ ]:
#Captute the one and only layer as its own object and reveal its object type
nlcd_layer = nlcd.layers[0]
type(nlcd_layer)

In [ ]:
#Show the layer
nlcd_layer

### Accesing image service layers directly, using their REST endpoint URL
What if you couldn't find the item's ID? If you can find the service's REST endpoint, you can create an image layer from that. 
 * First you have to import the `ImagerLayer` object from the ArcGIS Python API's `raster` submodule. 
 * Then we create the image layer from the REST URL. 
 * In this instance, we have to supply the `gis` object because layers in the Living Atlas are restricted, meaning you have to provide an authenticated GIS object. 

In [ ]:
#Import the ImageryLayer object into our session
from arcgis.raster import ImageryLayer

#Construct the layer directly from its URL
nlcd_layer2 = ImageryLayer(url='https://landscape10.arcgis.com/arcgis/rest/services/USA_NLCD_Land_Cover/ImageServer',
                        gis=gis)

### Exploring properties of the image layer
Before analyzing our image, let's look at how we access its properties

In [ ]:
#Show properties associated with the layer
list(nlcd_layer.properties.keys())

In [ ]:
#Show the fields
for f in nlcd_layer.properties.fields: print (f.name)

In [ ]:
#Reveal the item's spatial reference
nlcd_layer.properties.spatialReference

In [ ]:
#What are the pixel sizes in the X and Y direction? 
xPixelSize = nlcd_layer.properties.pixelSizeX
yPixelSize = nlcd_layer.properties.pixelSizeY
print(xPixelSize,yPixelSize)

In [ ]:
#Reveal the functions associated with the item
for fun in nlcd_layer.properties.rasterFunctionInfos: 
    print(fun.name,":",fun.description)

In [ ]:
#How many pixels can be downloaded at once? 
nlcd_layer.properties.maxRecordCount

### Mapping
Just as we did in previous notebooks, we can create a map widget and add this layer to it. 

In [ ]:
#Create the map, centered on Durham
m = gis.map("Durham County, NC")
#Add the nlcd_layer
m.add_layer(nlcd_layer)
#Show the map
m

In [ ]:
mapExtent = m.extent
mapExtent

## Subsetting our image
We cannot easily download this dataset in one chunk; it's too big, and that request would burden the server. Rather, we are limited by downloading 1000 pixels at time, and thus we need to constrain the extent of the data we download. The workflow here is as follows: 
* Create a geometry object to clip the raster at a management size.
 * Define an extent, either manually or using the [geocoding] module.
 * Convert it into an ArcGIS "geometry" object.
  * For this we will need to import the arcgis [geometry module](https://developers.arcgis.com/python/api-reference/arcgis.geometry.html) to conver the extent to an "envelope" and then into a "polygon".
* We'll then need to project this geometry to match the NLCD layer's spatial reference.
 * We'll need to know the image layer's spatial reference for this
* Then, we can [clip](https://developers.arcgis.com/python/api-reference/arcgis.raster.functions.html?arcgis.raster.functions.clip#clip) the NLCD layer using this geometry

Reference: https://developers.arcgis.com/python/guide/raster-analysis-advanced-concepts/

In [ ]:
#create a dictionary of extent values, in decimal degrees
myExtent = {'xmin': -79.15,
            'ymin': 35.95,
            'xmax': -78.95,
            'ymax': 36.15}

In [ ]:
#Alternative, create an extent using the API's geocoding module
from arcgis import geocoding
area = geocoding.geocode("Duke University",out_sr=nlcd_layer.properties.spatialReference)[0]
area['extent']

In [ ]:
#Set the area of interest of our NLCD image
nlcd_layer.extent=area['extent']

In [ ]:
#Display the image
nlcd_layer

In [ ]:
#Save the image
savedimg = nlcd_layer.export_image(bbox=area['extent'],
                        save_file='./DurhamNLCD.tif',
                        export_format='tiff',
                        compression_quality=100,
                        f='image'
                       )

In [ ]:
#Import the ArcGIS geometry module
from arcgis import geometry

In [ ]:
#Convert the dictionary to an "envelope"
envelope_DD = geometry.Envelope(myExtent)
#Convert the envelope into a polygon
extentPoly_DD = envelope_DD.polygon
#Show the polygon
extentPoly_DD

In [ ]:
extentPoly_DD.get_area(method='geodesic',units='meters')/900

In [ ]:
#Display the polygon on the map
map2 = gis.map("Durham Co, NC")
map2.draw(extentPoly_DD)
map2

In [ ]:
#Save the NLCD's spatial reference as a variable
nlcd_wkid = nlcd_layer.properties.spatialReference.wkid
nlcd_wkid

In [ ]:
#Project the extentPoly to the spatial reference of the imagery layer'
extentPoly_prj = geometry.project(geometries=[extentPoly_DD],
                                  in_sr=4326,
                                  out_sr=nlcd_sr)[0]
#Plot the projected polygon
extentPoly_prj

In [ ]:
#Import the clip function and clip the raster 
from arcgis.raster.functions import clip
lc_clip = clip(raster=nlcd_layer,
               geometry=extentPoly_prj)

In [ ]:
import pandas as pd
attDF = pd.DataFrame.from_records(nlcd_layer.attribute_table()['features'])
attDF = attDF['attributes'].apply(pd.Series).set_index('OBJECTID')
attDF.head()

In [ ]:
#Extract the extremes
xmin,ymin,xmax,ymax = extentPoly_prj.geoextent

In [ ]:
ext['xmin'] = xmin

In [ ]:
#Now we need to compute number of rows and columns contained in this extent
ext=area['extent']
xSize = int((ext['xmax'] - ext['xmin']) / 30)
ySize = int((ext['ymax'] - ext['ymin']) / 30)
xSize,ySize

* Use the imagery layer's [`export_image`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.raster.toc.html#arcgis.raster.ImageryLayer.export_image) function to extract a subset of data.

In [ ]:
#Extract data
img = nlcd_layer.export_image(bbox=ext,
                              size=[xSize,ySize],
                               f='image')
type(img)

In [ ]:
#Show the image
Image(img)

In [ ]:
#Or we can save the output directly to a file
savedimg = nlcd_layer.export_image(bbox=area['extent'],
                               save_folder='.', 
                               save_file='DurhamNLCD.tif',
                               size=[xSize,ySize],
                               export_format='tiff',
                               compression_quality=100,
                               f='image'
                              )

In [ ]:
#Or we can examine properties of our saved image as a JSON object
img_json = nlcd_layer.export_image(bbox=area['extent'],
                               f='json'
                              )
img_json